In [ ]:

import time
from huggingface_hub import HfApi
from dotenv import load_dotenv
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

In [ ]:


def section_by_section(input_text: str, tokenizer, model) -> str:
    print("🚀 섹션별 생성 시작...")
    
    sections = []
    
    # 각 섹션별 프롬프트
    section_prompts = [
        {
            "name": "프로젝트 상세 정보",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 다음 형식으로 프로젝트 상세 정보만 생성해주세요:\n**프로젝트 상세 정보:**\n{{완전한 딕셔너리 형태}}"
        },
        {
            "name": "관계 데이터", 
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 데이터베이스 관계 데이터만 생성해주세요:\n**관계 데이터:**\n[완전한 리스트 형태]"
        },
        {
            "name": "ERD 데이터",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 ERD 데이터만 생성해주세요:\n**ERD 데이터:**\n[완전한 리스트 형태]"
        },
        {
            "name": "API 명세",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 API 명세 데이터만 생성해주세요:\n**API 명세 데이터:**\n{{완전한 딕셔너리 형태}}"
        }
    ]
    
    for i, section in enumerate(section_prompts):
        start_time = time.time()
        print(f"[{i+1}/4] {section['name']} 생성 중...")
        
        inputs = tokenizer(section['prompt'], return_tensors='pt', truncation=True, max_length=512)
        eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')
        
        with torch.no_grad():
          outputs = model.generate(
              input_ids=inputs["input_ids"].to('cuda'),
              max_new_tokens=2048,        # 3072 → 2048 (적당히 줄이기)
              do_sample=False,            # True → False (그리디, 가장 빠름)
              use_cache=True,             # 캐시 사용
              num_beams=1,               # 빔 서치 끄기
              early_stopping=False,
              pad_token_id=tokenizer.eos_token_id,
              eos_token_id=None
          )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        section_content = response[len(section['prompt']):].strip()
        sections.append(section_content)
        
        elapsed = time.time() - start_time
        print(f"    ✅ 완료: {len(section_content)} 문자, {elapsed:.1f}초")
    
    # 모든 섹션 결합
    full_response = input_text + "\n\nAssistant: " + "\n\n".join(sections)
    print(f"🎉 전체 완료! 총 {len(full_response)} 문자 생성")
    return full_response

In [ ]:
api = HfApi()
username="Min-kyu"
MODEL_NAME = "PJA_LLM_MODEL_8bit"

load_dotenv()

api_key = os.getenv('HUG_API_KEY')

# 학습된 모델
tokenizer = AutoTokenizer.from_pretrained(f"{username}/{MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}")

# 추론
input_text = """
이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.
"""

inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to('cuda'),
        max_new_tokens=500,
        eos_token_id=eos_token_id,
        do_sample=True,
        temperature=1.0,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response.replace("', '", "',\n'").replace("}, {", "},\n{"))

In [2]:
from dotenv import load_dotenv
import os

load_dotenv()

api_key = os.getenv('HUG_API_KEY')

In [3]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

# 학습된 모델
tokenizer = AutoTokenizer.from_pretrained(f"{username}/{MODEL_NAME}")
model = AutoModelForCausalLM.from_pretrained(f"{username}/{MODEL_NAME}")

Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.
`low_cpu_mem_usage` was None, now set to True since model is quantized.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.


Assistant: **프로젝트 상세 정보:**
{'project_summary': {'title': '3인칭 카드게임 기반 MMORPG', 'category': '게임', 'target_users': ['게임 애호가', 'MMORPG 팬', '카드게임 유저'],'main_purpose': '시간이 10초 지날 때마다 하나씩 카드가 드롭되게하는 새로운 형식의 MMORPG 개발', 'key_features': [{'feature': '카드 드롭', 'description': '시간이 10초 지날 때마다 하나씩 카드가 드롭되는 기능'}, {'feature': '스토리', 'description': '사용자의 선택에 따라 드롭되는 카드의 형식이 달라지는 스토리라인 제공'}, {'feature': '선택 스토리라인', 'description': '사용자가 선택하는 스토리라인에 따라 카드의 형식이 달라짐'}, {'feature': '컨트롤 능력 강화', 'description': '랜덤 가챠과 순간적인 판단으로 사용자가 컨트롤하는 능력 증대'}], 'core_technologies': [{'category': 'Game Development', 'technologies': ['Unity', 'Unet', '2D/3D Animation']}, {'category': 'Game UI', 'technologies'

In [4]:
# 추론
input_text = """
이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.
"""

inputs = tokenizer(input_text, return_tensors='pt', truncation=True, max_length=512)
eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')

with torch.no_grad():
    outputs = model.generate(
        input_ids=inputs["input_ids"].to('cuda'),
        max_new_tokens=500,
        eos_token_id=eos_token_id,
        do_sample=True,
        temperature=1.0,
        pad_token_id=tokenizer.eos_token_id
    )
    
    response = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(response.replace("', '", "',\n'").replace("}, {", "},\n{"))


이 프로젝트는 3인칭 카드게임기반 MMORPG 게임 개발 프로젝트입니다. 창업을 목적으로 게임 데이터를 끌어다가 프로젝트를 하려고합니다. 주제는 간단하게 3인칭 카드게임을 mmorpg형식으로 만드려고 합니다. 전체적으로 어떻게 만들생각이냐면 시간이 10초 지날 때마다 하나씩 카드가 드롭되게 하는 형식으로 게임을 만들고싶습니다. 또한, 스토리가 있었으면 좋겠고 선택하는 스토리라인에 따라서 드롭되는 카드의 형식이 달랐으면 좋겠습니다. 이 게임을 통해서 사용자가 랜덤 가챠 + 순간적인 판단으로 컨트롤하는 능력이 늘었으면 좋겠다는 생각으로 프로젝트를 기획하였습니다.
사용 기술로는 Python, Node.js, Express, Django, Flask, Twilio, Kahoot, etc을 사용할 계획입니다. 데이터베이스는 MongoDB, FireSQL로 설계할 것이고 API로는 RESTful API, GraphQL API를 사용할 계획입니다. 기술적이고 실용적인 관점에서 프로젝트를 분석하며, 개발 시나리오를 빠르게 실행에 옮길 수 있는 구체적이고 실용적인 가치를 제공할겁니다.

Assistant: **프로젝트 상세 정보:**
{'project_summary': {'title': '3인칭 카드게임 기반 MMORPG 게임',
'category': 'MMORPG 게임',
'target_users': ['게임 애호가',
'카드게임 팬',
'대중 게임 사용자'],'main_purpose': '3인칭 카드게임을 mmorpg 형식으로 변환하여 사용자의 게임 경험을 증가시키기',
'key_features': [{'feature': '카드 드롭 시스템',
'description': '시간이 10초 지날 때마다 하나씩 카드가 드롭되는 형식'},
{'feature': '스토리',
'description': '선택하는 스토리라인에 따라 드롭되는 카드의 형식 달라짐'},
{'feature': '사용자 컨트롤',
'description': '랜드가 챠 기반으로 사용자가 실시간으로 판단 수행

In [9]:
import time

def section_by_section(input_text: str, tokenizer, model) -> str:
    print("🚀 섹션별 생성 시작...")
    
    sections = []
    
    # 각 섹션별 프롬프트
    section_prompts = [
        {
            "name": "프로젝트 상세 정보",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 다음 형식으로 프로젝트 상세 정보만 생성해주세요:\n**프로젝트 상세 정보:**\n{{완전한 딕셔너리 형태}}"
        },
        {
            "name": "관계 데이터", 
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 데이터베이스 관계 데이터만 생성해주세요:\n**관계 데이터:**\n[완전한 리스트 형태]"
        },
        {
            "name": "ERD 데이터",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 ERD 데이터만 생성해주세요:\n**ERD 데이터:**\n[완전한 리스트 형태]"
        },
        {
            "name": "API 명세",
            "prompt": f"{input_text}\n\n위 프로젝트에 대해 API 명세 데이터만 생성해주세요:\n**API 명세 데이터:**\n{{완전한 딕셔너리 형태}}"
        }
    ]
    
    for i, section in enumerate(section_prompts):
        start_time = time.time()
        print(f"[{i+1}/4] {section['name']} 생성 중...")
        
        inputs = tokenizer(section['prompt'], return_tensors='pt', truncation=True, max_length=512)
        eos_token_id = tokenizer.convert_tokens_to_ids('<|eot_id|>')
        
        with torch.no_grad():
          outputs = model.generate(
              input_ids=inputs["input_ids"].to('cuda'),
              max_new_tokens=2048,        # 3072 → 2048 (적당히 줄이기)
              do_sample=False,            # True → False (그리디, 가장 빠름)
              use_cache=True,             # 캐시 사용
              num_beams=1,               # 빔 서치 끄기
              early_stopping=False,
              pad_token_id=tokenizer.eos_token_id,
              eos_token_id=None
          )
        
        response = tokenizer.decode(outputs[0], skip_special_tokens=True)
        section_content = response[len(section['prompt']):].strip()
        sections.append(section_content)
        
        elapsed = time.time() - start_time
        print(f"    ✅ 완료: {len(section_content)} 문자, {elapsed:.1f}초")
    
    # 모든 섹션 결합
    full_response = input_text + "\n\nAssistant: " + "\n\n".join(sections)
    print(f"🎉 전체 완료! 총 {len(full_response)} 문자 생성")
    return full_response

In [10]:
result = section_by_section(input_text, tokenizer, model)

🚀 섹션별 생성 시작...
[1/4] 프로젝트 상세 정보 생성 중...


KeyboardInterrupt: 